# Flights in the United States: 2004 - 2008
## by Maleina Bidek

## Preliminary Wrangling

In this notebook, we will be exploring a dataset that contains flight arrival and departure details for all commercial flights within the United States, including whether or not flights were delayed or cancelled and what the reason was. The data set is from the American Statistical Society and can be found [here.](https://community.amstat.org/jointscsg-section/dataexpo/dataexpo2009) While the original set contains data from October 1987 to December 2008, in this analysis, we will specifically be exploring data over the five year period from 2004 through 2008.

*Note that while the data set's home page says that it contains records through April 2008, we have seen that it actually contains records through December 2008.*

In [1]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

The first thing we need to do is load in our datasets. Each is contained in a .csv file that has been compressed. Because all of the files are structed identically, i.e. they all have the same columns and data types, we can import them sucessively and append them to the same dataframe for further inspection. These sets are quite large, so this could take a while!

In [2]:
# Read in first data set 
print("Loading 2004 data...")
df_flights = pd.read_csv('data/2004.csv.bz2',compression='bz2')

# Construct list of successive years that need to be imported
years = np.arange(2005, 2009, 1)

# Loop through remaining years
for year in years:
    # Because loading may take a while, print which dataset we're loading
    print("Loading " + year.astype(str) + " data...")
    filename = "data/" + year.astype(str) + ".csv.bz2"
    df_temp = pd.read_csv(filename, compression='bz2')
    df_flights = df_flights.append(df_temp)
    
print("Loading complete!")

Loading 2004 data...
Loading 2005 data...
Loading 2006 data...
Loading 2007 data...
Loading 2008 data...
Loading complete!


Next, we'll check to see if we have issues with our data that require cleaning. First, however, let's check the size of our dataset.

In [3]:
df_flights.shape

(35874731, 29)

We have over 35 million rows and 29 columns. This is a huge set! One of the things we'll need to explore is paring down the set so that computations take less time. Let's do a quick visual check on the set first. Again, since we haven't pared down our set, this could take a little while.

In [4]:
df_flights

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2004,1,12,1,623.0,630,901.0,915,UA,462,...,7.0,11.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
1,2004,1,13,2,621.0,630,911.0,915,UA,462,...,16.0,16.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
2,2004,1,14,3,633.0,630,920.0,915,UA,462,...,4.0,15.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
3,2004,1,15,4,627.0,630,859.0,915,UA,462,...,4.0,10.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
4,2004,1,16,5,635.0,630,918.0,915,UA,462,...,3.0,13.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7009723,2008,12,13,6,1002.0,959,1204.0,1150,DL,1636,...,6.0,45.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
7009724,2008,12,13,6,834.0,835,1021.0,1023,DL,1637,...,5.0,23.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
7009725,2008,12,13,6,655.0,700,856.0,856,DL,1638,...,24.0,12.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
7009726,2008,12,13,6,1251.0,1240,1446.0,1437,DL,1639,...,13.0,13.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN


One of the first things that sticks out is that `Year`, `Month` and `DayofMonth` could be combined into a single date field. However, because we don't anticipate conducting a time series analysis, i.e. plotting flight information over time, it actually will be more useful to have these columns separated. This will make it easier for us to aggregate over months or days of the week, for example.

There are also a number of fields that we don't need: `DepTime`, `ArrTime`, `TailNum`, `ActualElapsedTime`, `CRSElapsedTime`, `AirTime`, `Distance`, `TaxiIn`, `TaxiOut`. We can remove these.

`CarrierDelay`, `WeatherDelay`, `NASDelay`, `SecurityDelay`, and `LateArrCraftDelay` are columns that are supposed to represent the amount of time spent on that particular type of delay. They are not mutually exclusive: It is possible to have an amount listed in more than one of these columns for the same flight. Note that in this report I generally refer to them as the 'delay reason' columns. Given that we already see that there are null values in some of these fields, we'll want to explore further what to do with these columns.

Because this set is so large, we will first remove the columns that we won't be using. However, just before we start modifying the dataframe, we'll save a new copy.

In [5]:
# Make a copy of our dataset so that we won't have to reload everything 
# if we run into problems cleaning
df_flights_clean = df_flights.copy()

In [6]:
# Remove unneeded columns
df_flights_clean.drop(['DepTime', 'ArrTime', 'TailNum', 'ActualElapsedTime', 
                 'CRSElapsedTime', 'AirTime', 'Distance', 'TaxiIn', 'TaxiOut'], 
                axis=1, inplace=True)

In [7]:
# Ensure that columns have been removed.
df_flights_clean.columns

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'CRSDepTime', 'CRSArrTime',
       'UniqueCarrier', 'FlightNum', 'ArrDelay', 'DepDelay', 'Origin', 'Dest',
       'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'],
      dtype='object')

Let's check our data types next, noting that we'll want to reduce unnessary storage where possible.

In [10]:
df_flights_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35874731 entries, 0 to 7009727
Data columns (total 20 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Year               int64  
 1   Month              int64  
 2   DayofMonth         int64  
 3   DayOfWeek          int64  
 4   CRSDepTime         int64  
 5   CRSArrTime         int64  
 6   UniqueCarrier      object 
 7   FlightNum          int64  
 8   ArrDelay           float64
 9   DepDelay           float64
 10  Origin             object 
 11  Dest               object 
 12  Cancelled          int64  
 13  CancellationCode   object 
 14  Diverted           int64  
 15  CarrierDelay       float64
 16  WeatherDelay       float64
 17  NASDelay           float64
 18  SecurityDelay      float64
 19  LateAircraftDelay  float64
dtypes: float64(7), int64(9), object(4)
memory usage: 5.6+ GB


Some data types are incorrect. `CRSDepTime` and `CRSArrTime` should be converted to time. `Diverted` and `Cancelled` appear to use 1 for yes and 0 for no and could easily be converted to the more useful Boolean type.  Before we do that let's double check that `Diverted` and `Cancelled` have only the values of 1 (yes) and 0 (no).

In [11]:
df_flights_clean['Diverted'].value_counts()

0    35796289
1       78442
Name: Diverted, dtype: int64

In [12]:
df_flights_clean['Cancelled'].value_counts()

0    35193128
1      681603
Name: Cancelled, dtype: int64

We can see that these two colums have no other values besides zero and one which means that they can easily be converted.

Before we can fix any of our data types, we'll need to check for nulls, as the presence of these could prevent the converstion. Due to the size of our set, this information wasn't present when we called the info() fuction. We'll have to check explicitly.

In [13]:
# Search all columns for nulls and print the number of them.
for row in df_flights_clean.columns:
    print('{}: {}'.format(row, sum(df_flights_clean[row].isna())))

Year: 0
Month: 0
DayofMonth: 0
DayOfWeek: 0
CRSDepTime: 0
CRSArrTime: 0
UniqueCarrier: 0
FlightNum: 0
ArrDelay: 760045
DepDelay: 680415
Origin: 0
Dest: 0
Cancelled: 0
CancellationCode: 35193119
Diverted: 0
CarrierDelay: 5484993
WeatherDelay: 5484993
NASDelay: 5484993
SecurityDelay: 5484993
LateAircraftDelay: 5484993


We will need to investigate the nulls in the `ArrDelay`, `DepDelay`, `CancellationCode`, `CarrierDelay`, `WeatherDelay`, `NASDelay`, `SecurityDelay`, and `LateAircraftDelay` columns.

Let's explore `ArrDelay` and `DepDelay` first.

In [14]:
# Seach for records where ArrDelay is null
df_flights_clean[df_flights_clean['ArrDelay'].isna()]

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ArrDelay,DepDelay,Origin,Dest,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
12,2004,1,26,1,630,915,UA,462,NaN,NaN,ORD,CLT,1,B,0,0.0,0.0,0.0,0.0,0.0
21,2004,1,4,7,1505,1725,UA,463,NaN,NaN,ORD,PDX,1,C,0,0.0,0.0,0.0,0.0,0.0
23,2004,1,6,2,1505,1725,UA,463,NaN,NaN,ORD,PDX,1,B,0,0.0,0.0,0.0,0.0,0.0
24,2004,1,7,3,1510,1732,UA,463,NaN,NaN,ORD,PDX,1,C,0,0.0,0.0,0.0,0.0,0.0
25,2004,1,8,4,1510,1732,UA,463,NaN,NaN,ORD,PDX,1,B,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7009455,2008,12,13,6,600,815,DL,1211,NaN,NaN,LGA,CVG,1,A,0,NaN,NaN,NaN,NaN,NaN
7009464,2008,12,13,6,1930,2129,DL,1218,NaN,NaN,CVG,LGA,1,A,0,NaN,NaN,NaN,NaN,NaN
7009564,2008,12,13,6,700,1035,DL,1421,NaN,NaN,BOS,FLL,1,A,0,NaN,NaN,NaN,NaN,NaN
7009565,2008,12,13,6,1115,1432,DL,1422,NaN,NaN,FLL,BOS,1,A,0,NaN,NaN,NaN,NaN,NaN


These flights all appear to be cancellations. Let's confirm this by seeing if any nulls exist when canceled is false.

In [15]:
df_flights_clean[(df_flights_clean['ArrDelay'].isna()) & (df_flights_clean['Cancelled']!=1)]

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ArrDelay,DepDelay,Origin,Dest,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
280,2004,1,18,7,725,933,UA,469,NaN,73.0,MSY,LAX,0,NaN,1,0.0,0.0,0.0,0.0,0.0
435,2004,1,4,7,1950,2212,UA,473,NaN,196.0,ORD,SNA,0,NaN,1,0.0,0.0,0.0,0.0,0.0
2190,2004,1,9,5,2120,2340,UA,519,NaN,-2.0,ORD,SEA,0,NaN,1,0.0,0.0,0.0,0.0,0.0
2292,2004,1,3,6,1845,1910,UA,521,NaN,18.0,ICT,DEN,0,NaN,1,0.0,0.0,0.0,0.0,0.0
2887,2004,1,24,6,1445,1711,UA,535,NaN,34.0,ORD,DFW,0,NaN,1,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7006200,2008,12,10,3,640,859,DL,1610,NaN,34.0,JAN,ATL,0,NaN,1,NaN,NaN,NaN,NaN,NaN
7006401,2008,12,11,4,1106,1950,DL,26,NaN,169.0,LAX,JFK,0,NaN,1,NaN,NaN,NaN,NaN,NaN
7007034,2008,12,11,4,1520,1708,DL,1102,NaN,7.0,IAD,ATL,0,NaN,1,NaN,NaN,NaN,NaN,NaN
7007389,2008,12,11,4,725,945,DL,1491,NaN,-8.0,LGA,MSY,0,NaN,1,NaN,NaN,NaN,NaN,NaN


OK, it looks like a number of these are also diversions. Let's check to see if there are any nulls that are not cancellations nor diversions.

In [16]:
df_flights_clean[(df_flights_clean['ArrDelay'].isna()) & (df_flights_clean['Cancelled']!=1) & (df_flights_clean['Diverted']!=1)]

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ArrDelay,DepDelay,Origin,Dest,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay


No rows are returned. It appears that the `ArrDelay` column is null only in the case where the flight was cancelled or diverted, which seems correct. We can fill these with 0's to be consistent with other flights in which there was no arrival delay.

In [17]:
# Remove the nulls
df_flights_clean['ArrDelay'].fillna(0, inplace=True)

In [18]:
# Ensure that the nulls have been removed. This should return 0.
sum(df_flights_clean[df_flights_clean['ArrDelay'].isna()].count())

0

Let's check the `DepDelay` column now.

In [19]:
# Seach for records where DepDelay is null
df_flights_clean[df_flights_clean['DepDelay'].isna()]

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ArrDelay,DepDelay,Origin,Dest,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
12,2004,1,26,1,630,915,UA,462,0.0,NaN,ORD,CLT,1,B,0,0.0,0.0,0.0,0.0,0.0
21,2004,1,4,7,1505,1725,UA,463,0.0,NaN,ORD,PDX,1,C,0,0.0,0.0,0.0,0.0,0.0
23,2004,1,6,2,1505,1725,UA,463,0.0,NaN,ORD,PDX,1,B,0,0.0,0.0,0.0,0.0,0.0
24,2004,1,7,3,1510,1732,UA,463,0.0,NaN,ORD,PDX,1,C,0,0.0,0.0,0.0,0.0,0.0
25,2004,1,8,4,1510,1732,UA,463,0.0,NaN,ORD,PDX,1,B,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7009455,2008,12,13,6,600,815,DL,1211,0.0,NaN,LGA,CVG,1,A,0,NaN,NaN,NaN,NaN,NaN
7009464,2008,12,13,6,1930,2129,DL,1218,0.0,NaN,CVG,LGA,1,A,0,NaN,NaN,NaN,NaN,NaN
7009564,2008,12,13,6,700,1035,DL,1421,0.0,NaN,BOS,FLL,1,A,0,NaN,NaN,NaN,NaN,NaN
7009565,2008,12,13,6,1115,1432,DL,1422,0.0,NaN,FLL,BOS,1,A,0,NaN,NaN,NaN,NaN,NaN


These results are similar to what we saw the `ArrDelay` column. Let's check if these are all a result of the flight being cancelled.

In [20]:
df_flights_clean[(df_flights_clean['DepDelay'].isna()) & (df_flights_clean['Cancelled']!=1)]

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ArrDelay,DepDelay,Origin,Dest,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay


OK, there are no rows returned, which means that if `DepDelay` is null, then the flight was cancelled. This makes sense. Again, we should fill these with 0's for consistency.

In [21]:
# Replace nulls with 0
df_flights_clean['DepDelay'].fillna(0, inplace=True)

In [22]:
# Verify that the nulls have been removed. This should return 0.
sum(df_flights_clean[df_flights_clean['DepDelay'].isna()].count())

0

Next, let's check those nulls in the `CancellationCode` column and see if any occur if cancelled is true (i.e, its 1).

In [23]:
# Check if any nulls exist when the flight was cancelled
df_flights_clean[(df_flights_clean['CancellationCode'].isna()) &
          (df_flights_clean['Cancelled'] == 1)]

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ArrDelay,DepDelay,Origin,Dest,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay


No, the nulls are only present when the flight has not been cancelled. We can actually leave these alone as there should be no be no cancellation code unless the flight has been cancelled. Also, since this field will not have its type changed these nulls won't be a problem.

Finally, let's check the nulls in the `CarrierDelay`, `WeatherDelay`, `NASDelay`, `SecurityDelay`, and `LateAircraftDelay` columns. Let's check if any of these columns are null when there is an arrival or departure delay, i.e. either of the values of `DepDelay` and `ArrDelay` are positive.

In [24]:
# Check if any of the delay reason columns is null when the flight has a 
# positive number of arrival or departure delay minutes.
df_flights_clean[((df_flights_clean['CarrierDelay'].isna()) | 
          (df_flights_clean['WeatherDelay'].isna()) |
          (df_flights_clean['NASDelay'].isna()) |
          (df_flights_clean['SecurityDelay'].isna()) |
          (df_flights_clean['LateAircraftDelay'].isna())) & 
          ((df_flights_clean['ArrDelay'] > 0)|
          (df_flights_clean['DepDelay'] > 0))]

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ArrDelay,DepDelay,Origin,Dest,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,1955,2225,WN,335,-14.0,8.0,IAD,TPA,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,735,1000,WN,3231,2.0,19.0,IAD,TPA,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,620,750,WN,448,14.0,8.0,IND,BWI,0,NaN,0,NaN,NaN,NaN,NaN,NaN
5,2008,1,3,4,1915,2110,WN,378,11.0,25.0,IND,JAX,0,NaN,0,NaN,NaN,NaN,NaN,NaN
8,2008,1,3,4,615,650,WN,11,2.0,2.0,IND,MCI,0,NaN,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7009720,2008,12,13,6,800,1026,DL,1633,6.0,-4.0,MSY,ATL,0,NaN,0,NaN,NaN,NaN,NaN,NaN
7009722,2008,12,13,6,750,859,DL,1636,2.0,-1.0,SAV,ATL,0,NaN,0,NaN,NaN,NaN,NaN,NaN
7009723,2008,12,13,6,959,1150,DL,1636,14.0,3.0,ATL,IAD,0,NaN,0,NaN,NaN,NaN,NaN,NaN
7009726,2008,12,13,6,1240,1437,DL,1639,9.0,11.0,IAD,ATL,0,NaN,0,NaN,NaN,NaN,NaN,NaN


Here we have a number of flights that were actually delayed but no reason was noted. It looks like all of these have arrival delays of less than 15 minutes, however. As a general rule of thumb, 15 minutes is the threshold for calling a flight delayed. Let's check if all arrival delays over 15 minutes have values in these delay reason columns.

In [25]:
# Check if there are nulls in the delay reason columns when 
# a flight has an arrival delay of at least 15 mintues
df_flights_clean[((df_flights_clean['CarrierDelay'].isna()) | 
          (df_flights_clean['WeatherDelay'].isna()) |
          (df_flights_clean['NASDelay'].isna()) |
          (df_flights_clean['SecurityDelay'].isna()) |
          (df_flights_clean['LateAircraftDelay'].isna())) & 
          (df_flights_clean['ArrDelay'] >= 15)]

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ArrDelay,DepDelay,Origin,Dest,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay


We can see that all flights with an arrival delay of 15 minutes or more do not have nulls in any of the delay reason columns. Let's check if it is possible to have values in those columns if the arrival delay is less than 15 minutes, or will they all be null?

In [26]:
# Check if there are values in the delay reason columns when 
# a flight has an arrival delay of less than 15 mintues
df_flights_clean[~((df_flights_clean['CarrierDelay'].isna()) & 
          (df_flights_clean['WeatherDelay'].isna()) &
          (df_flights_clean['NASDelay'].isna()) &
          (df_flights_clean['SecurityDelay'].isna()) &
          (df_flights_clean['LateAircraftDelay'].isna())) & 
          (df_flights_clean['ArrDelay'] < 15)]

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ArrDelay,DepDelay,Origin,Dest,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2004,1,12,1,630,915,UA,462,-14.0,-7.0,ORD,CLT,0,NaN,0,0.0,0.0,0.0,0.0,0.0
1,2004,1,13,2,630,915,UA,462,-4.0,-9.0,ORD,CLT,0,NaN,0,0.0,0.0,0.0,0.0,0.0
2,2004,1,14,3,630,915,UA,462,5.0,3.0,ORD,CLT,0,NaN,0,0.0,0.0,0.0,0.0,0.0
3,2004,1,15,4,630,915,UA,462,-16.0,-3.0,ORD,CLT,0,NaN,0,0.0,0.0,0.0,0.0,0.0
4,2004,1,16,5,630,915,UA,462,3.0,5.0,ORD,CLT,0,NaN,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7453207,2007,12,15,6,1335,1855,DL,48,1.0,6.0,SLC,CVG,0,NaN,0,0.0,0.0,0.0,0.0,0.0
7453208,2007,12,15,6,1645,1836,DL,49,-9.0,1.0,CVG,SLC,0,NaN,0,0.0,0.0,0.0,0.0,0.0
7453209,2007,12,15,6,730,909,DL,55,9.0,-1.0,CVG,ATL,0,NaN,0,0.0,0.0,0.0,0.0,0.0
7453210,2007,12,15,6,1605,1736,DL,58,13.0,-7.0,MCO,ATL,0,NaN,0,0.0,0.0,0.0,0.0,0.0


So it is possible that flights with arrival delays of less than 15 minutes will have a value in at least one of the delay reason columns, but we've also seen that they may not. A more interesting result of this query is that there are some records that have a negative departure delay and a negative arrival delay and they don't have nulls. All of the departure reason columns have 0's instead. Given that this means that a non-delayed flight can have 0's in these columns instead of nulls, then for consistency it makes sense to fill our nulls in all of these columns with 0's.

In [27]:
# Remove nulls from each of the delay reason columns 
delay_reason_cols = ['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
for col in delay_reason_cols:
    df_flights_clean[col].fillna(0, inplace=True)

In [28]:
# Ensure that the columns have no more nulls. All should return 0
for col in delay_reason_cols:
    print('# of nulls for '+ col + ': {}'.format(sum(df_flights_clean[df_flights_clean[col].isna()].count())))

# of nulls for CarrierDelay: 0
# of nulls for WeatherDelay: 0
# of nulls for NASDelay: 0
# of nulls for SecurityDelay: 0
# of nulls for LateAircraftDelay: 0


Next, let's finally fix our data types. `Diverted` and `Cancelled` will be converted to Boolean. Other fields will be reduced from 64-bits where possible. We'll address `CRSDepTime` and `CRSArrTime` shortly as their conversion is a bit more complicated. 

In [30]:
# Convert all floats and integers to 16 bit (instead of 32)
# Convert Cancelled and Diverted to boolean
df_flights_clean['Year'] = df_flights_clean['Year'].astype(np.int16)
df_flights_clean['Month'] = df_flights_clean['Month'].astype(np.int16)
df_flights_clean['DayofMonth'] = df_flights_clean['DayofMonth'].astype(np.int16)
df_flights_clean['DayOfWeek'] = df_flights_clean['DayOfWeek'].astype(np.int16)
df_flights_clean['DayOfWeek'] = df_flights_clean['DayOfWeek'].astype(np.int16)
df_flights_clean['FlightNum'] = df_flights_clean['FlightNum'].astype(np.int16)
df_flights_clean['ArrDelay'] = df_flights_clean['ArrDelay'].astype(np.float16)
df_flights_clean['DepDelay'] = df_flights_clean['DepDelay'].astype(np.float16)
df_flights_clean['Cancelled'] = df_flights_clean['Cancelled'].astype(bool)
df_flights_clean['Diverted'] = df_flights_clean['Diverted'].astype(bool)
df_flights_clean['CarrierDelay'] = df_flights_clean['CarrierDelay'].astype(np.float16)
df_flights_clean['WeatherDelay'] = df_flights_clean['WeatherDelay'].astype(np.float16)
df_flights_clean['NASDelay'] = df_flights_clean['NASDelay'].astype(np.float16)
df_flights_clean['SecurityDelay'] = df_flights_clean['SecurityDelay'].astype(np.float16)
df_flights_clean['LateAircraftDelay'] = df_flights_clean['LateAircraftDelay'].astype(np.float16)

In [31]:
# Verify that our data types have been converted
df_flights_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35874731 entries, 0 to 7009727
Data columns (total 20 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Year               int16  
 1   Month              int16  
 2   DayofMonth         int16  
 3   DayOfWeek          int16  
 4   CRSDepTime         int64  
 5   CRSArrTime         int64  
 6   UniqueCarrier      object 
 7   FlightNum          int16  
 8   ArrDelay           float16
 9   DepDelay           float16
 10  Origin             object 
 11  Dest               object 
 12  Cancelled          bool   
 13  CancellationCode   object 
 14  Diverted           bool   
 15  CarrierDelay       float16
 16  WeatherDelay       float16
 17  NASDelay           float16
 18  SecurityDelay      float16
 19  LateAircraftDelay  float16
dtypes: bool(2), float16(7), int16(5), int64(2), object(4)
memory usage: 2.7+ GB


Next, lets convert our `CRSDepTime` and `CRSArrTime` columns to dates. We'll actually convert them to the timedelta type, as this will allow us to perform calculations with them.

In [32]:
# Convert the columns by using floor division to extract the hour
# and mod division to extract the minutes. Both the hour and the minutes
# are converted to timedelta format and then added together to get the full time in proper format
df_flights_clean['CRSDepTime'] = (pd.to_timedelta(df_flights_clean['CRSDepTime']//100, unit='h') + 
                            pd.to_timedelta(df_flights_clean['CRSDepTime']%100, unit='m'))
df_flights_clean['CRSArrTime'] = (pd.to_timedelta(df_flights_clean['CRSArrTime']//100, unit='h') + 
                            pd.to_timedelta(df_flights_clean['CRSArrTime']%100, unit='m'))

In [33]:
# Verify the data types are correct
df_flights_clean[['CRSDepTime', 'CRSArrTime']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35874731 entries, 0 to 7009727
Data columns (total 2 columns):
 #   Column      Dtype          
---  ------      -----          
 0   CRSDepTime  timedelta64[ns]
 1   CRSArrTime  timedelta64[ns]
dtypes: timedelta64[ns](2)
memory usage: 821.1 MB


Finally, let's check for duplicates.

In [34]:
sum(df_flights_clean.duplicated())

60

There are very few duplicates, however, we will still remove these from our set. 

In [35]:
# Drop duplicates
df_flights_clean.drop_duplicates(inplace=True)

In [36]:
# Check that the number of duplicates is now 0.
sum(df_flights_clean.duplicated())

0

Let's take a quick look at our stats.

In [37]:
df_flights_clean.describe()

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,FlightNum,ArrDelay,DepDelay,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,3.587467e+07,3.587467e+07,3.587467e+07,3.587467e+07,35874671,35874671,3.587467e+07,35874671.0,35874671.0,35874671.0,35874671.0,35874671.0,35874671.0,35874671.0
mean,2.006002e+03,6.497322e+00,1.573346e+01,3.939221e+00,0 days 13:30:47.420250348,0 days 15:10:17.673372391,2.146726e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,1.408290e+00,3.421688e+00,8.790446e+00,1.989782e+00,0 days 04:38:47.520875743,0 days 04:48:10.866856352,1.946000e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,2.004000e+03,1.000000e+00,1.000000e+00,1.000000e+00,0 days 00:00:00,0 days 00:00:00,1.000000e+00,-1302.0,-1200.0,0.0,0.0,-49.0,0.0,0.0
25%,2.005000e+03,4.000000e+00,8.000000e+00,2.000000e+00,0 days 09:30:00,0 days 11:18:00,5.930000e+02,-9.0,-4.0,0.0,0.0,0.0,0.0,0.0
50%,2.006000e+03,7.000000e+00,1.600000e+01,4.000000e+00,0 days 13:25:00,0 days 15:20:00,1.484000e+03,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,2.007000e+03,9.000000e+00,2.300000e+01,6.000000e+00,0 days 17:20:00,0 days 19:09:00,3.363000e+03,11.0,8.0,0.0,0.0,0.0,0.0,0.0
max,2.008000e+03,1.200000e+01,3.100000e+01,7.000000e+00,0 days 23:59:00,1 days 00:00:00,9.912000e+03,2598.0,2600.0,2580.0,1510.0,1392.0,533.0,1366.0


Let's also get a final count of rows and columns.

In [38]:
df_flights_clean.shape

(35874671, 20)

As we progressed through the assessment, we didn't find any other quality issues to be addressed. Therefore, let's save a copy of our cleaned dataset.

In [39]:
df_flights_final = df_flights_clean.copy()

### What is the structure of your dataset?

As we can see above, our cleaned dataset contains 35,874,671 rows and 20 columns. Most of columns are of a numeric type: `Year`, `Month`, `DayofMonth`, `DayofWeek` and `FlightNum` are discrete integers, the first four being categorical. `ArrDelay`, `DepDelay`, `CarrierDelay`, `WeatherDelay`, `NASDelay`, `SecurityDelay`, `LateAircraftDelay` are all continuous float types.

`UniqueCarrier`, `Origin`, `Dest` and `CancellationCode` are categorical text fields. `Cancelled` and `Diverted` have been converted to boolean type. Finally, `CRSDepTime` and `CRSArrTime` have been converted to time types (specifically, timedelta type).

### What is/are the main feature(s) of interest in your dataset?

We're going to be exploring whether flights departed and arrived on-time or if there were delays and/or cancellations. In particular, are there patterns by time of day, time of year, origin and destination? Are particular carriers or flights particularly prone to issues?

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

All of the columns that we've retainted will be of interest, but these in particular:
- `ArrDelay`
- `DepDelay`
- `CarrierDelay`
- `WeatherDelay`
- `NASDelay`
- `SecurityDelay`
- `LateAircraftDelay` 
- `Cancelled`
- `CancellationCode`
- `Diverted`
- `Month`
- `DayofWeek`
- `CRSDepTime`
- `CRSArrTime`
- `UniqueCarrier`
- `FlightNum`
- `Origin`
- `Dest`

## Univariate Exploration

> In this section, investigate distributions of individual variables. If
you see unusual points or outliers, take a deeper look to clean things up
and prepare yourself to look at relationships between variables.

> Make sure that, after every plot or related series of plots, that you
include a Markdown cell with comments about what you observed, and what
you plan on investigating next.

### Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?

> Your answer here!

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> Your answer here!

## Bivariate Exploration

> In this section, investigate relationships between pairs of variables in your
data. Make sure the variables that you cover here have been introduced in some
fashion in the previous section (univariate exploration).

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> Your answer here!

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> Your answer here!

## Multivariate Exploration

> Create plots of three or more variables to investigate your data even
further. Make sure that your investigations are justified, and follow from
your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

> Your answer here!

### Were there any interesting or surprising interactions between features?

> Your answer here!

> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!